<a href="https://colab.research.google.com/github/jkcg-learning/SuperGradients/blob/main/SuperGradients_Kaggle_Template_Image_Folder_Format.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src='https://github.com/Deci-AI/super-gradients/blob/master/documentation/assets/SG_img/SG%20-%20Horizontal%20Glow.png?raw=true'>


## If you want to learn about some fundamental  model architectures, be sure to check out [my FREE courses on Udemy](https://www.udemy.com/user/harpreet-sahota-4/).

In this notebook, you'll use the SuperGradients training library to perform image classifcation! 

[SuperGradients](https://github.com/Deci-AI/super-gradients) is an open-source PyTorch based training library that has a number of pre-trained models for you to use, training recipies that will get you amazing accuracy, and many [training tricks](https://www.deeplearningdaily.community/t/tips-for-training-your-neural-networks/307) that you can use with just the "flip of a switch". For this example you'll use an EfficientNetB0 to perfom the classification. You can check out our [model zoo](https://github.com/Deci-AI/super-gradients/blob/master/src/super_gradients/training/Computer_Vision_Models_Pretrained_Checkpoints.md) and use any of the pretrained models we have available.

Feel free to reach out to me on my community forum, [Deep Learning Daily (free and open to all)](https://www.deeplearningdaily.community/), should you have any questions.



# Image Classification Project Template with SuperGradients

This notebook template is for data that is already in ImageFolder format.

If you are unfamiliar with ImageFolder format, it looks like this:

```
├── train
│   ├── class1
|      ├── 1.jpg
│      ├── 2.jpg
│   ├── class2
|      ├── 1.jpg
│      ├── 2.jpg
├── valid
│   ├── class1
|      ├── 1.jpg
│      ├── 2.jpg
│   ├── class2
|      ├── 1.jpg
│      ├── 2.jpg
```

With parent folder repeating for validtaion and testing data.

In [ ]:
%%capture 
# After installation is complete you must restart the notebook otherwise you 
# will face some import errors
!pip install imutils
!pip install super_gradients==3.0.7
!pip install albumentations 
!pip install split-folders[full]

In [ ]:
import os
import math
import random
from typing import Dict, List,Tuple
import requests

import numpy as np
import matplotlib.pyplot as plt
import glob
from pathlib import Path, PurePath
import pathlib
import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import datasets
from torchvision import transforms

from PIL import Image

import albumentations as A
from albumentations.pytorch import ToTensorV2

from sklearn.model_selection import train_test_split

from imutils import paths

import splitfolders
import textwrap

import super_gradients
from super_gradients.common.object_names import Models
from super_gradients.training import Trainer
from super_gradients.training import training_hyperparams
from super_gradients.training.metrics.classification_metrics import Accuracy, Top5
from super_gradients.training.utils.early_stopping import EarlyStop
from super_gradients.training import models
from super_gradients.training.utils.callbacks import Phase

# Number of classes

Simple utility to fetch the number of classes.

In [ ]:
from pathlib import Path

def count_subdirectories(path: str) -> int:
    """
    Counts the number of subdirectories in the given directory path.
    """
    dir_path = Path(path)
    subdirectories = [f for f in dir_path.iterdir() if f.is_dir()]
    return len(subdirectories)

# Example usage
parent_dir = "/path/to/parent/directory"
num_subdirectories = count_subdirectories(parent_dir)
print(f"Number of subdirectories in {parent_dir}: {num_subdirectories}")


# Config 

This holds variables for the notebook.

You will define the model, training params, image type, number of classes, and 
relevant directories in this class.



If you have a question you can leave a comment on this notebook, or visit the community and post it in the [Q&A section](https://www.deeplearningdaily.community/c/qanda/8).

## Use a different pretrained model

You can change the model you use. Take a look at the [SG model zoo](https://github.com/Deci-AI/super-gradients/blob/master/src/super_gradients/training/Computer_Vision_Models_Pretrained_Checkpoints.md)

For example, if you wanted to use RegNet you would do the following:

```
MODEL_NAME = "regnetY800"
```

Note you can also pass 'model_name=regnetY200', 'model_name=regnetY400', 'model_name=regnetY600' to try a variety of the architecture

For ResNet50, you would do:

```
MODEL_NAME="resnet50" 
```

Note you can also pass 'model_name=resnet18' or 'model_name=resnet34' to try a variety of the architecture

For MobileNetV2, you would do:

```
MODEL_NAME='mobilenet_v2'
```

For MobileNetV3, you would do:

```
MODEL_NAME='mobilenet_v3_large'
```

Note you can also pass 'model_name=mobilenet_v3_small' to try a variety of the architecture


For ViT, you would do:


```
MODEL_NAME='vit_base'
```

Note you can also pass 'model_name=vit_large' to try a variety of the architecture

In [ ]:
class config:
    # specify the paths to datasets
    ROOT_DIR = Path()
    TRAIN_DIR = ROOT_DIR.joinpath('train')
    TEST_DIR = ROOT_DIR.joinpath('test')
    VAL_DIR = ROOT_DIR.joinpath('val')

    # set the input height and width
    INPUT_HEIGHT = 224
    INPUT_WIDTH = 224

    # set the input heig/ht and width
    IMAGENET_MEAN = [0.485, 0.456, 0.406]
    IMAGENET_STD = [0.229, 0.224, 0.225]
    
    IMAGE_TYPE = '.jpg'
    BATCH_SIZE = 128
    MODEL_NAME = 
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    TRAINING_PARAMS = 'training_hyperparams/default_train_params'
    
    NUM_CLASSES = num_subdirectories

    
    CHECKPOINT_DIR = 'checkpoints'


# Split data into training, validation and testing sets.

If your data is not already split up into train, validation and test you can use this.

Note the `ratio=(.8, .1, .1)` argument is will split it into 80% train, 10% validation, and 10% split. You can adjust thee as you wish. If you only need to split into two folders (say you already have train and validation and just need to split the validation into a test set) then just use two floats instead.


In [ ]:
splitfolders.ratio(config.DOWNLOAD_DIR, output=config.OUTPUT_DIR, seed=42, ratio=(.8, .1, .1), group_prefix=None, move=False)

# Plot random images

In [ ]:
train_image_path_list = list(sorted(paths.list_images(config.TRAIN_DIR)))
train_image_path_sample = random.sample(population=train_image_path_list, k=20)

def examine_images(images:list):
    num_images = len(images)
    num_rows = int(math.ceil(num_images/5))
    num_cols = 5
    
    fig, axs = plt.subplots(num_rows, num_cols, figsize=(30, 30),tight_layout=True)
    axs = axs.ravel()

    for i, image_path in enumerate(images[:num_images]):
        image = Image.open(image_path)
        label = PurePath(image_path).parent.name
        axs[i].imshow(image)
        axs[i].set_title(f"Class: {label}", fontsize=40)
        axs[i].axis('off')
    plt.show()

examine_images(train_image_path_sample)

# Check for class distribution


In [ ]:
# Get a list of all subdirectories in the root directory
subdirs = [d for d in Path(config.TRAIN_DIR).iterdir() if d.is_dir()]

# Initialize a dictionary to store the count of images in each subdirectory
image_count = {}

# Iterate through each subdirectory
for subdir in subdirs:
    subdir_images = list(sorted(paths.list_images(subdir)))
    image_count[subdir.name] = len(subdir_images)

plt.bar(image_count.keys(), image_count.values())
# add the count numbers on top of the bars
for i, (subdir, count) in enumerate(image_count.items()):
    plt.text(i, count + 3, str(count), ha='center')

# set the title and labels for the plot
plt.title("Number of Images in Each Subdirectory")
plt.xlabel("Subdirectories")
plt.ylabel("Counts")

# show the plot
plt.show()

# Augmentations


In [ ]:
# initialize our data augmentation functions
resize = transforms.Resize(size=(config.INPUT_HEIGHT,config.INPUT_WIDTH))
make_tensor = transforms.ToTensor()
normalize = transforms.Normalize(mean=config.IMAGENET_MEAN, std=config.IMAGENET_STD)
center_cropper = transforms.CenterCrop((config.INPUT_HEIGHT,config.INPUT_WIDTH))
random_horizontal_flip = transforms.RandomHorizontalFlip(p=0.75)
random_vertical_flip = transforms.RandomVerticalFlip(p=0.75)
random_rotation = transforms.RandomRotation(degrees=90)
random_crop = transforms.RandomCrop(size=(200,200))
augmix = transforms.AugMix(severity = 3, mixture_width=3, alpha=0.2)
auto_augment = transforms.AutoAugment()
random_augment = transforms.RandAugment()

# initialize our training and validation set data augmentation pipeline
train_transforms = transforms.Compose([
  resize, 
  auto_augment,
  augmix,
  random_augment,
  make_tensor,
  normalize
])

val_transforms = transforms.Compose([resize, make_tensor, normalize])

# Show what one image looks like after augmentation

In [ ]:
def apply_transform(img: Image, transform) -> np.ndarray:
    """
    Applies a transform to a PIL Image and returns a numpy array of the transformed image.

    Args:
        img (PIL.Image): The input image to transform.
        transform (torchvision.transforms.Compose): The transform to apply to the image.

    Returns:
        np.ndarray: A numpy array representing the transformed image.
    """
    # Apply the transform to the image
    if isinstance(transform, torchvision.transforms.Compose):
        # Apply PyTorch transform to image array
        transformed_image = train_transforms(img)

    elif isinstance(transform, A.Compose):
        # Apply Albumentations transform to image array
        img_array = np.array(img)
        transformed_image = transform(image=img_array)["image"]

    # Convert the image tensor to a numpy array and transpose the axes to (height, width, channels)
    img_array = transformed_image.numpy().transpose((1, 2, 0))

    # Clip the pixel values to the range [0, 1]
    img_array = np.clip(img_array, 0, 1)

    return img_array


def visualize_transform(image: np.ndarray, original_image: np.ndarray = None) -> None:
    """
    Visualize the transformed image.

    Args:
        image (np.ndarray): A NumPy array representing the transformed image.
        original_image (np.ndarray, optional): A NumPy array representing the original image. Defaults to None.
    """
    fontsize = 18
    
    if original_image is None:
        # Create a plot with 1 row and 2 columns.
        f, ax = plt.subplots(1, 2, figsize=(12, 12))

        # Show the transformed image in the first column.
        ax[0].imshow(image)
    else:
        # Create a plot with 1 row and 2 columns.
        f, ax = plt.subplots(1, 2, figsize=(12, 12))

        # Show the original image in the first column.
        ax[0].imshow(original_image)
        ax[0].set_title('Original image', fontsize=fontsize)
        
        # Show the transformed image in the second column.
        ax[1].imshow(image)
        ax[1].set_title('Transformed image', fontsize=fontsize)
        
img = Image.open(random.choice(train_image_path_list))
img_array = apply_transform(img, train_transforms)
visualize_transform(img_array, original_image=img)

# Datasets and Dataloadrer

In [ ]:
def create_dataloaders(
    train_dir: str, 
    val_dir: str,
    test_dir: str,
    train_transform: transforms.Compose,
    val_transform:  transforms.Compose,
    test_transform:  transforms.Compose,
    batch_size: int, 
    num_workers: int=2
):
  """Creates training and validation DataLoaders.
  Args:
    train_dir: Path to training data.
    val_dir: Path to validation data.
    transform: Transformation pipeline.
    batch_size: Number of samples per batch in each of the DataLoaders.
    num_workers: An integer for number of workers per DataLoader.
  Returns:
    A tuple of (train_dataloader, val_dataloader, class_names).
  """
  # Use ImageFolder to create dataset
  train_data = datasets.ImageFolder(train_dir, transform=train_transform)
  val_data = datasets.ImageFolder(val_dir, transform=val_transform)
  test_data = datasets.ImageFolder(test_dir, transform=val_transform)  

  print(f"[INFO] training dataset contains {len(train_data)} samples...")
  print(f"[INFO] validation dataset contains {len(val_data)} samples...")
  print(f"[INFO] test dataset contains {len(test_data)} samples...")

  # Get class names
  class_names = train_data.classes
  print(f"[INFO] dataset contains {len(class_names)} labels...")

  # Turn images into data loaders
  print("[INFO] creating training and validation set dataloaders...")
  train_dataloader = DataLoader(
      train_data,
      batch_size=batch_size,
      shuffle=True,
      drop_last=True,
      num_workers=num_workers,
      pin_memory=True,
      persistent_workers=True
  )
  val_dataloader = DataLoader(
      val_data,
      batch_size=batch_size,
      shuffle=False,
      num_workers=num_workers,
      pin_memory=True,
      drop_last=False,
      persistent_workers=True
  )

  test_dataloader = DataLoader(
      test_data,
      batch_size=batch_size,
      shuffle=False,
      num_workers=num_workers,
      pin_memory=True,
      drop_last=False,
      persistent_workers=True
  )

  return train_dataloader, val_dataloader, test_dataloader, class_names

In [ ]:
train_dataloader, valid_dataloader, test_dataloader, class_names = create_dataloaders(train_dir=config.TRAIN_DIR,
                                                                     val_dir=config.VAL_DIR,
                                                                     test_dir=config.TEST_DIR,
                                                                     train_transform=train_transforms,
                                                                     val_transform=val_transforms,
                                                                     test_transform=val_transforms,
                                                                     batch_size=config.BATCH_SIZE)

NUM_CLASSES = len(class_names)

# Training Params

Learn more about SuperGradients training params [here](https://github.com/Deci-AI/super-gradients/blob/master/tutorials/what_are_recipes_and_how_to_use.ipynb)

There are a few things you can try to see how you fare: try a different optimizer (you can the optimizer by using a passing of the following strings "Adam", "AdamW", "SGD", or "RMSProp".

You can also enable exponential moving average (change the param for "ema" to True), you can enable zero weight decay on bias and batch norm (change the parm for "zero_weight_decay_on_bias_and_bn" to True), or you can change the level of label smoothing by playing around with the 'smooth_eps' value.

Get a detailed description of all training parameters, what they mean, and their allowable values [here](https://github.com/Deci-AI/super-gradients/blob/master/src/super_gradients/recipes/training_hyperparams/default_train_params.yaml).

In [ ]:
training_params =  training_hyperparams.get(config.TRAINING_PARAMS)

In [ ]:
# To reduce clutter in the notebook I've turned the verbosity off, you can turn it on to see the full output
early_stop_acc = EarlyStop(Phase.VALIDATION_EPOCH_END, monitor="Accuracy", mode="max", patience=7, verbose=False)
early_stop_val_loss = EarlyStop(Phase.VALIDATION_EPOCH_END, monitor="LabelSmoothingCrossEntropyLoss", mode="min", patience=7, verbose=False)

training_params["train_metrics_list"] = [Accuracy(), Top5()]
training_params["valid_metrics_list"] = [Accuracy(), Top5()]
training_params["phase_callbacks"] = [early_stop_acc, early_stop_val_loss]

# Set the silent mode to True to reduce clutter in the notebook, you can turn it on to see the full output
training_params["silent_mode"] = False
# We'll turn off the use of exponential moving average and zero weight decay on bias and batch norm
training_params['zero_weight_decay_on_bias_and_bn'] = False
training_params["optimizer"] = 'Adam'
training_params["criterion_params"] = {'smooth_eps': 0.20}
training_params["max_epochs"] = 250
training_params["initial_lr"] = 0.0001

# Get model



In [ ]:
model = models.get(config.MODEL_NAME, num_classes = config.NUM_CLASSES, pretrained_weights='imagenet')

# Instantiate trainer

In [ ]:
full_model_trainer = Trainer(experiment_name='0_Baseline_Experiment', ckpt_root_dir=config.CHECKPOINT_DIR)

# Train model

In [ ]:
full_model_trainer.train(model=model, 
              training_params=training_params, 
              train_loader=train_dataloader,
              valid_loader=valid_dataloader)

# Get best model

In [ ]:
best_full_model = models.get(config.MODEL_NAME,
                        num_classes=config.NUM_CLASSES,
                        checkpoint_path=os.path.join(full_model_trainer.checkpoints_dir_path, "ckpt_best.pth"))

# note if you're averaging the best model then replace "ckpt_best.pth" with "average_model.pth"

# Evaluate on test set

In [ ]:
full_model_trainer.test(model=best_full_model,
            test_loader=test_dataloader,
            test_metrics_list=['Accuracy', 'Top5'])

# Plot predictions

In [ ]:
import requests
def pred_and_plot_image(image_path: str, 
                        subplot: Tuple[int, int, int],  # subplot tuple for `subplot()` function
                        class_names: List[str] = class_names,
                        model: torch.nn.Module = best_full_model,
                        image_size: Tuple[int, int] = (config.INPUT_HEIGHT, config.INPUT_WIDTH),
                        transform: torchvision.transforms = None,
                        device: torch.device=config.DEVICE):

    if isinstance(image_path, pathlib.PosixPath):
      img = Image.open(image_path)
    else: 
      img = Image.open(requests.get(image_path, stream=True).raw)

    # create transformation for image (if one doesn't exist)
    if transform is None:
        transform = transforms.Compose([
            transforms.Resize(image_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=config.IMAGENET_MEAN,
                                 std=config.IMAGENET_STD),
        ])
    transformed_image = transform(img)

    # make sure the model is on the target device
    model.to(device)

    # turn on model evaluation mode and inference mode
    model.eval()
    with torch.inference_mode():
        # add an extra dimension to image (model requires samples in [batch_size, color_channels, height, width])
        transformed_image = transformed_image.unsqueeze(dim=0)

        # make a prediction on image with an extra dimension and send it to the target device
        target_image_pred = model(transformed_image.to(device))

    # convert logits -> prediction probabilities (using torch.softmax() for multi-class classification)
    target_image_pred_probs = torch.softmax(target_image_pred, dim=1)

    # convert prediction probabilities -> prediction labels
    target_image_pred_label = torch.argmax(target_image_pred_probs, dim=1)

    # actual label
    ground_truth = PurePath(image_path).parent.name

    # plot image with predicted label and probability 
    plt.subplot(*subplot)
    plt.imshow(img)
    if isinstance(image_path, pathlib.PosixPath):
        title = f"Ground Truth: {ground_truth} | Pred: {class_names[target_image_pred_label]} | Prob: {target_image_pred_probs.max():.3f}"
    else:
        title = f"Pred: {class_names[target_image_pred_label]} | Prob: {target_image_pred_probs.max():.3f}"
    plt.title("\n".join(textwrap.wrap(title, width=20)))  # wrap text using textwrap.wrap() function
    plt.axis(False)
    

def plot_random_test_images(model):
    num_images_to_plot = 30
    test_image_path_list = [pathlib.PosixPath(p) for p in sorted(paths.list_images(config.TEST_DIR))] # get list all image paths from test data 
    test_image_path_sample = random.sample(population=test_image_path_list, # go through all of the test image paths
                                           k=num_images_to_plot) # randomly select 'k' image paths to pred and plot

    # set up subplots
    num_rows = int(np.ceil(num_images_to_plot / 5))
    fig, ax = plt.subplots(num_rows, 5, figsize=(15, num_rows * 3))
    ax = ax.flatten()

    # Make predictions on and plot the images
    for i, image_path in enumerate(test_image_path_sample):
        pred_and_plot_image(model=model, 
                            image_path=image_path,
                            class_names=class_names,
                            subplot=(num_rows, 5, i+1),  # subplot tuple for `subplot()` function
                            image_size=(config.INPUT_HEIGHT, config.INPUT_WIDTH))

    # adjust spacing between subplots
    plt.subplots_adjust(wspace=1)
    plt.show()

In [ ]:
plot_random_test_images(best_full_model)


# Predict on images from internet

In [ ]:
image_url = 
pred_and_plot_image(image_path= image_url, subplot=(1, 1, 1))

# Confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix

# Set model to evaluation mode
best_full_model.eval()

# Create empty lists to store true labels and predicted labels
true_labels = []
predicted_labels = []

# Loop over batches in test dataloader, make predictions, and append true and predicted labels to lists
for images, labels in test_dataloader:
    images = images.to(config.DEVICE)
    labels = labels.to(config.DEVICE)
    with torch.no_grad():
        outputs = best_full_model(images)
        _, predicted = torch.max(outputs.data, 1)
    true_labels.extend(labels.cpu().numpy())
    predicted_labels.extend(predicted.cpu().numpy())

# Calculate confusion matrix, precision, and recall
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Create figure and axis objects with larger size and font size
fig, ax = plt.subplots(figsize=(12, 10))
plt.rcParams.update({'font.size': 16})

# Create heatmap of confusion matrix
im = ax.imshow(conf_matrix, cmap='Blues')

# Add colorbar to heatmap
cbar = ax.figure.colorbar(im, ax=ax)

# Set tick labels and axis labels with larger font size
ax.set_xticks(np.arange(len(class_names)))
ax.set_yticks(np.arange(len(class_names)))
ax.set_xticklabels(class_names, fontsize=14)
ax.set_yticklabels(class_names, fontsize=14)
ax.set_xlabel('Predicted label', fontsize=16)
ax.set_ylabel('True label', fontsize=16)

# Rotate tick labels and set alignment
plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")

# Add text annotations to heatmap
for i in range(len(class_names)):
    for j in range(len(class_names)):
        if conf_matrix[i, j] >= -1:  # Modify threshold value as needed
            text = ax.text(j, i, conf_matrix[i, j],
                           ha="center", va="center", color="y", fontsize=16)
        else:
            text = ax.text(j, i, "",
                           ha="center", va="center", color="y")

# Add title to plot with larger font size
ax.set_title("Confusion matrix", fontsize=20)

# Show plot
plt.show()

# Your homework

Copy/fork this notebook and try some different architectures.

If you have a question you can leave a comment on this notebook, or visit the community and post it in the [Q&A section](https://www.deeplearningdaily.community/c/qanda/8).

## Use a different pretrained model

You can change the model you use. Take a look at the [SG model zoo](https://github.com/Deci-AI/super-gradients/blob/master/src/super_gradients/training/Computer_Vision_Models_Pretrained_Checkpoints.md)

For example, if you wanted to use RegNet you would do the following:

```
resnet_imagenet_model = models.get(model_name='regnetY800', num_classes=NUM_CLASSES, pretrained_weights='imagenet)
resnet_params =  training_hyperparams.get('training_hyperparams/imagenet_regnetY_train_params')
```

Note you can also pass 'model_name=regnetY200', 'model_name=regnetY400', 'model_name=regnetY600' to try a variety of the architecture

For ResNet50, you would do:

```
resnet_imagenet_model = models.get(model_name='resnet50', num_classes=NUM_CLASSES, pretrained_weights='imagenet)
resnet_params =  training_hyperparams.get('training_hyperparams/imagenet_resnet50_train_params')
```

Note you can also pass 'model_name=resnet18' or 'model_name=resnet34' to try a variety of the architecture

For MobileNetV2, you would do:

```
mobilenet_imagenet_model = models.get(model_name='mobilenet_v2', num_classes=NUM_CLASSES, pretrained_weights='imagenet)
resnet_params =  training_hyperparams.get('training_hyperparams/imagenet_mobilenetv2_train_params')
```

For MobileNetV3, you would do:

```
mobilenet_imagenet_model = models.get(model_name='mobilenet_v3_large', num_classes=NUM_CLASSES, pretrained_weights='imagenet)
resnet_params =  training_hyperparams.get('training_hyperparams/imagenet_mobilenetv3_train_params')
```

Note you can also pass 'model_name=mobilenet_v3_small' to try a variety of the architecture


For ViT, you would do:


```
vit_imagenet_model = models.get(model_name='vit_base', num_classes=NUM_CLASSES, pretrained_weights='imagenet')
vit_params =  training_hyperparams.get("training_hyperparams/imagenet_vit_train_params")
```

Note you can also pass 'model_name=vit_large' to try a variety of the architecture


I encourage you play around with different optimizers, all you have to do is change the value of `training_params["optimizer"]`. You can use one of ['Adam','SGD','RMSProp'] out of the box. You can play around with the optimizer params as well.

In general, play and tweak around the training recipies...

## Training recipes

SuperGradients has a number of [training recipes](https://github.com/Deci-AI/super-gradients/tree/master/src/super_gradients/recipes) you can use. [See here](https://github.com/Deci-AI/super-gradients/blob/master/src/super_gradients/recipes/training_hyperparams/default_train_params.yaml) for more information about the training params.

If you're using Weights and Biases to track your experiments, you would do the following

```
sg_logger: wandb_sg_logger
sg_logger_params:
project_name: <YOUR PROJECT NAME>
entity: algo
api_server: https://wandb.research.deci.ai
save_checkpoints_remote: True
save_tensorboard_remote: True
save_logs_remote: True
```